In [ ]:
import numpy as np
import pandas as pd
import requests
import csv
import os
import re
import string
from openai import OpenAI
import math
import json
import tiktoken

: 

In [ ]:
def load_file(file_path, degree_level, degree_name):
    file_name = os.path.basename(file_path)
    file_name = file_name.lower()
    if file_path:
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
        except UnicodeDecodeError:
            with open(file_path, "r", encoding="utf-16-le") as file:
                text = file.read()
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")
        num_tokens = len(encoding.encode(text))
        if num_tokens > 15000:
            number_of_needed_files = math.ceil(num_tokens / 4000)
            chunk_size = math.ceil(num_tokens / number_of_needed_files)
            chunks = [
                text[i : i + chunk_size] for i in range(0, num_tokens, chunk_size)
            ]
            print(number_of_needed_files)
            return chunks, num_tokens, file_name, degree_name, degree_level

        return text, num_tokens, file_name, degree_name, degree_level

: 

In [ ]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


def label_file(text, num_tokens, degree_level, degree):
    if num_tokens > 4000:
        num_tokens = 2000

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": "You are an assistant who extracts questions and the corresponding answers from texts to create a dataset for a machine learning model, you return only valid .jsonl as well as all questions and answers in english.",
            },
            {
                "role": "user",
                "content": f'Create as many relevant questions and the corresponding answers for this text: {text}. Return each question-answer pair in the following format: {{"conversations": [<insert question here> , <insert answer here>], "source":{degree_level+"_"+degree}}}',
            },
        ],
        temperature=0.2,
        max_tokens=num_tokens,
        top_p=0.1,
    )
    response_ = response.choices[0].message.content
    return response_

: 

In [20]:
# Specify the directory you want to start from
rootDir = "/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)"

file_paths = []

degree = os.path.basename(rootDir)

if degree.startswith("b"):
    degree_level = "Bachelor"
else:
    degree_level = "Master"
degree_name = degree[2:]

print(degree_name)
print(degree_level)

for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        # os.path.join() method in Python join one or more path components intelligently.
        file_paths.append(os.path.join(dirName, fname))
for f in file_paths:
    print(f)
for file in file_paths:
    text, num_tokens, file_name, degree, degree_level = load_file(
        file, degree_level, degree_name
    )
    print(num_tokens)
    if num_tokens > 16000:
        for t in text:
            print("t:", len(t))
            text = label_file(t, num_tokens, degree, degree_level)
    else:
        text = label_file(text, num_tokens, degree, degree_level)

    with open(degree + "V2" + ".jsonl", "a") as f:
        # Write the string to the file
        f.write(text)

Advanced_Systems_Design_(Systemtechnik)
Master
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/.DS_Store
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/data_m_Advanced_Systems_Design_(Systemtechnik)_main_page.txt
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/data_m_Advanced_Systems_Design_(Systemtechnik)_info_table.csv
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/_Studium_Generale_Prozess_in_der_Fakult_t_Elektronik_und_Informatik.pdf.txt
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/_m_Advanced_Systems_Design_(Systemtechnik)_staff.txt
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems_Design_(Systemtechnik)/_Zeugn_Festl.Wahlf_30_MSD_20210426.pdf.txt
/Users/patrickmuller/Documents/dev/05_Project/01_data/m_Advanced_Systems

In [12]:
# just for testing can be removed later
# TODO was just for testing can be removed later
from datasets import load_dataset
from typing import List, Optional


def format_dataset(
    dataset_partition: dict, include_multi_turn_conversations: bool
) -> List[dict]:
    formatted_ds = []

    for entry in dataset_partition:
        convo = entry["conversations"]
        if include_multi_turn_conversations:
            for i in range(0, len(convo) - 1, 2):
                formatted_ds.append(
                    {"instruction": convo[i], "input": "", "output": convo[i + 1]}
                )

        else:
            formatted_ds.append(
                {"instruction": convo[0], "input": "", "output": convo[1]}
            )

    return formatted_ds


dataset = load_dataset(
    path="/Users/patrickmuller/Documents/dev/05_Project/02_code/dataset_converter",
    trust_remote_code=True,
)
train_data = format_dataset(dataset["train"], False)

Generating train split: 3194 examples [00:00, 58920.11 examples/s]
Generating test split: 11 examples [00:00, 5183.97 examples/s]
